In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)

In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:

def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:
#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
df


,instruction,output,tokens
0,words food crapilicious,4,"[words, food, crapilicious]"
1,words food crapilicious,4,"[words, food, crapilicious]"
2,white,4,[white]
3,classy whore red velvet cupcakes,4,"[classy, whore, red, velvet, cupcakes]"
4,meh p thanks heads concerned another angry dud...,4,"[meh, p, thanks, heads, concerned, another, an..."
...,...,...,...
2951,u tell ppl ok someone ridiculed ur race cuz so...,2,"[u, tell, ppl, ok, someone, ridiculed, ur, rac..."
2952,right battle dumb fuck cuz whites see nigger e...,2,"[right, battle, dumb, fuck, cuz, whites, see, ..."
2953,mad rt fuck obama dumb ass nigger,2,"[mad, rt, fuck, obama, dumb, ass, nigger]"
2954,fuck dumb nigger would never stand fuckin show...,2,"[fuck, dumb, nigger, would, never, stand, fuck..."


In [ ]:
from datasets import load_dataset
import pandas as pd
import sklearn

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df

,instruction,output,tokens
0,words food crapilicious,4,"[words, food, crapilicious]"
1,words food crapilicious,4,"[words, food, crapilicious]"
2,white,4,[white]
3,classy whore red velvet cupcakes,4,"[classy, whore, red, velvet, cupcakes]"
4,meh p thanks heads concerned another angry dud...,4,"[meh, p, thanks, heads, concerned, another, an..."
...,...,...,...
2951,u tell ppl ok someone ridiculed ur race cuz so...,2,"[u, tell, ppl, ok, someone, ridiculed, ur, rac..."
2952,right battle dumb fuck cuz whites see nigger e...,2,"[right, battle, dumb, fuck, cuz, whites, see, ..."
2953,mad rt fuck obama dumb ass nigger,2,"[mad, rt, fuck, obama, dumb, ass, nigger]"
2954,fuck dumb nigger would never stand fuckin show...,2,"[fuck, dumb, nigger, would, never, stand, fuck..."


In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Making x_train and y_train columns
X_train = train_df['instruction']
y_train = train_df['output']

In [ ]:
#Making y_test columns
X_test = test_df['instruction']
y_test = test_df['output']

In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenization and Padding
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
# Tokenization and Padding
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
# Load pre-trained GloVe word embeddings
def load_glove_embedding(file_path):
    embeddings_index = {}
    with open('/content/drive/MyDrive/glove.6B.100d.txt', encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

file_path = '/content/drive/MyDrive/glove.6B.100d.txt'
glove_embeddings_index = load_glove_embedding(file_path)

In [ ]:
# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim)) # max_words is the maximum number of words to consider
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = glove_embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
#Train Glove Embeddings using gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

# Convert GloVe format to Word2Vec format
glove_file = '/content/drive/MyDrive/glove.6B.100d.txt'
tmp_file = get_tmpfile("/content/drive/MyDrive/glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, tmp_file)

# Train GloVe embeddings
model = Word2Vec(X_train, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

# Access trained word vectors
word_vectors = model.wv

<ipython-input-32-5fce192b02b1>:9: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


In [ ]:
#Train Glove Embeddings using gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

# Convert GloVe format to Word2Vec format
glove_file = '/content/drive/MyDrive/glove.6B.100d.txt'
tmp_file = get_tmpfile("/content/drive/MyDrive/glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, tmp_file)

# Train GloVe embeddings
model = Word2Vec(X_test, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

# Access trained word vectors
word_vectors = model.wv

<ipython-input-33-c6382b3d8b3d>:9: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to convert tokenized text data into sequences of GloVe word vectors
def text_to_glove_vectors(text_data, word_vectors):
    glove_vectors = []
    for text in text_data:
        vector = [word_vectors[word] for word in text if word in word_vectors]
        if vector:
            glove_vectors.append(np.mean(vector, axis=0))  # Use average of word vectors
        else:
            glove_vectors.append(np.zeros(word_vectors.vector_size))  # Use zero vector if no word found
    return np.array(glove_vectors)

# Convert tokenized text data to GloVe vectors
X_train_glove = text_to_glove_vectors(X_train, word_vectors)
X_test_glove = text_to_glove_vectors(X_test, word_vectors)

In [ ]:
# Train logistic regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_glove, y_train)

# Make predictions
logreg_train_preds = logreg_model.predict(X_train_glove)

# Calculate evaluation metrics
accuracy = accuracy_score(y_train, logreg_train_preds)
precision = precision_score(y_train, logreg_train_preds, average='weighted')
recall = recall_score(y_train, logreg_train_preds, average='weighted')
f1 = f1_score(y_train, logreg_train_preds, average='weighted')

print("Logistic Regression Test Scores:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Logistic Regression Test Scores:
Accuracy: 0.20854483925549916
Precision: 0.2777233239453782
Recall: 0.20854483925549916
F1-Score: 0.07643154388002073


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train logistic regression model - Test Data
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_test_glove, y_test)

# Make predictions
logreg_test_preds = logreg_model.predict(X_test_glove)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, logreg_test_preds)
precision = precision_score(y_test, logreg_test_preds, average='weighted')
recall = recall_score(y_test, logreg_test_preds, average='weighted')
f1 = f1_score(y_test, logreg_test_preds, average='weighted')

print("Logistic Regression Test Scores:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Logistic Regression Test Scores:
Accuracy: 0.21114864864864866
Precision: 0.2111772305300224
Recall: 0.21114864864864866
F1-Score: 0.07599626724626725


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train SVM model - Train Data
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(X_train_glove, y_train)

# Make predictions
svm_train_preds = svm_model.predict(X_train_glove)

# Calculate evaluation metrics
accuracy_svm = accuracy_score(y_train, svm_train_preds)
precision_svm = precision_score(y_train, svm_train_preds, average='weighted')
recall_svm = recall_score(y_train, svm_train_preds, average='weighted')
f1_svm = f1_score(y_train, svm_train_preds, average='weighted')

print("SVM Test Scores:")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-Score:", f1_svm)

SVM Test Scores:
Accuracy: 0.20769881556683586
Precision: 0.10791471866411358
Recall: 0.20769881556683586
F1-Score: 0.07477349370683065


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#SVM on Test Data
svm_model1 = SVC()
svm_model1.fit(X_test_glove, y_test)

# Make predictions
svm_test_preds = svm_model1.predict(X_test_glove)

# Calculate evaluation metrics
accuracy_svm1 = accuracy_score(y_test, svm_test_preds)
precision_svm1 = precision_score(y_test, svm_test_preds, average='weighted')
recall_svm1 = recall_score(y_test, svm_test_preds, average='weighted')
f1_svm1 = f1_score(y_test, svm_test_preds, average='weighted')

print("SVM Test Scores:")
print("Accuracy:", accuracy_svm1)
print("Precision:", precision_svm1)
print("Recall:", recall_svm1)
print("F1-Score:", f1_svm1)

SVM Test Scores:
Accuracy: 0.21114864864864866
Precision: 0.2111772305300224
Recall: 0.21114864864864866
F1-Score: 0.07599626724626725


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train Neural Network model - Train Data
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_train_glove, y_train)

# Make predictions on test data
nn_train_preds = nn_model.predict(X_train_glove)

# Calculate evaluation metrics
accuracy_nn = accuracy_score(y_train, nn_train_preds)
precision_nn = precision_score(y_train, nn_train_preds, average='weighted')
recall_nn = recall_score(y_train, nn_train_preds, average='weighted')
f1_nn = f1_score(y_train, nn_train_preds, average='weighted')

print("Neural Network Test Scores:")
print("Accuracy:", accuracy_nn)
print("Precision:", precision_nn)
print("Recall:", recall_nn)
print("F1-Score:", f1_nn)


Neural Network Test Scores:
Accuracy: 0.3223350253807107
Precision: 0.4060580173024843
Recall: 0.3223350253807107
F1-Score: 0.27035643603847126


In [ ]:
# Train Neural Network model - Test Data
nn_model1 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model1.fit(X_test_glove, y_test)

# Make predictions on test data
nn_test_preds = nn_model1.predict(X_test_glove)

# Calculate evaluation metrics
accuracy_nn = accuracy_score(y_test, nn_test_preds)
precision_nn = precision_score(y_test, nn_test_preds, average='weighted')
recall_nn = recall_score(y_test, nn_test_preds, average='weighted')
f1_nn = f1_score(y_test, nn_test_preds, average='weighted')

print("Neural Network Test Scores:")
print("Accuracy:", accuracy_nn)
print("Precision:", precision_nn)
print("Recall:", recall_nn)
print("F1-Score:", f1_nn)

Neural Network Test Scores:
Accuracy: 0.20945945945945946
Precision: 0.04387326515704894
Recall: 0.20945945945945946
F1-Score: 0.07255020383512005


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#LSTM - Train Data
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define maximum sequence length and maximum number of words
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

# Tokenization and Padding for LSTM
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_lstm = tokenizer.texts_to_sequences(X_train)
X_train_lstm = pad_sequences(X_train_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)


# Make predictions on train data
lstm_train_preds = (lstm_model.predict(X_train_lstm) > 0.5).astype(int).flatten()

# Calculate evaluation metrics
accuracy_lstm = accuracy_score(y_train, lstm_train_preds)
precision_lstm = precision_score(y_train, lstm_train_preds, average='weighted')
recall_lstm = recall_score(y_train, lstm_train_preds, average='weighted')
f1_lstm = f1_score(y_train, lstm_train_preds, average='weighted')

print("LSTM Train Scores:")
print("Accuracy:", accuracy_lstm)
print("Precision:", precision_lstm)
print("Recall:", recall_lstm)
print("F1-Score:", f1_lstm)


Epoch 1/10
74/74 [==============================] - 6s 44ms/step - loss: -7.1912 - accuracy: 0.1434
Epoch 2/10
74/74 [==============================] - 5s 70ms/step - loss: -22.9121 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 3s 44ms/step - loss: -31.7262 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 4s 50ms/step - loss: -39.2102 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 4s 49ms/step - loss: -46.2908 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 5s 66ms/step - loss: -53.1662 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 3s 45ms/step - loss: -59.9294 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 3s 44ms/step - loss: -66.6006 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 4s 50ms/step - loss: -73.2210 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 1s 12ms/step
LSTM Train Scores:
Accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#LSTM - Test Data
# Define maximum sequence length and maximum number of words
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

# Tokenization and Padding for LSTM
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_test)

# Tokenization and Padding for test data
X_test_lstm = tokenizer.texts_to_sequences(X_test)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model1 = Sequential()
lstm_model1.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model1.add(LSTM(32))
lstm_model1.add(Dense(1, activation='sigmoid'))

# Compile and test LSTM model
lstm_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model1.fit(X_test_lstm, y_test, epochs=10, batch_size=32)


# Make predictions on train data
lstm_test_preds = (lstm_model1.predict(X_test_lstm) > 0.5).astype(int).flatten()

# Calculate evaluation metrics
accuracy_lstm1 = accuracy_score(y_test, lstm_test_preds)
precision_lstm1 = precision_score(y_test, lstm_test_preds, average='weighted')
recall_lstm1 = recall_score(y_test, lstm_test_preds, average='weighted')
f1_lstm1 = f1_score(y_test, lstm_test_preds, average='weighted')

print("LSTM Train Scores:")
print("Accuracy:", accuracy_lstm1)
print("Precision:", precision_lstm1)
print("Recall:", recall_lstm1)
print("F1-Score:", f1_lstm1)


Epoch 1/10
19/19 [==============================] - 3s 45ms/step - loss: 0.2656 - accuracy: 0.1301
Epoch 2/10
19/19 [==============================] - 1s 45ms/step - loss: -4.9854 - accuracy: 0.1385
Epoch 3/10
19/19 [==============================] - 1s 45ms/step - loss: -11.5402 - accuracy: 0.1385
Epoch 4/10
19/19 [==============================] - 1s 69ms/step - loss: -15.7727 - accuracy: 0.1385
Epoch 5/10
19/19 [==============================] - 1s 74ms/step - loss: -19.1596 - accuracy: 0.1385
Epoch 6/10
19/19 [==============================] - 1s 76ms/step - loss: -21.7915 - accuracy: 0.1385
Epoch 7/10
19/19 [==============================] - 1s 64ms/step - loss: -24.0893 - accuracy: 0.1385
Epoch 8/10
19/19 [==============================] - 1s 44ms/step - loss: -26.1952 - accuracy: 0.1385
Epoch 9/10
19/19 [==============================] - 1s 44ms/step - loss: -28.1912 - accuracy: 0.1385
Epoch 10/10
19/19 [==============================] - 1s 12ms/step
LSTM Train Scores:
Accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
